# IMPORTS

In [1]:
# !pip install transformers datasets scikit-learn pandas openvino onnx nncf
# !pip install accelerate

In [2]:
import multiprocessing as mp
mp.set_start_method("spawn", force=True)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import torch
from tqdm.auto import tqdm

/home/stepan/Documents/PythonProject/Binary_classifier_of_reviews_on_IMDB/venv_classifier/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings
warnings.filterwarnings('ignore')

# DATA

In [3]:
# для проверки пайплайна
dataset = load_dataset("imdb")
train_df = pd.DataFrame(dataset['train']).sample(10000, random_state=42).reset_index(drop=True)
test_df = pd.DataFrame(dataset['test']).sample(2000, random_state=42).reset_index(drop=True)

In [6]:
train_df.to_csv('../datasets/train.csv', sep=';', index=False, encoding='utf-8')
test_df.to_csv('../datasets/test.csv', sep=';', index=False, encoding='utf-8')

In [6]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_length=512, text_col="text", label_col="label"):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.text_col = text_col
        self.label_col = label_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.loc[idx, self.text_col]
        label = self.df.loc[idx, self.label_col]

        tokens = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
  
        item = {key: val.squeeze() for key, val in tokens.items()}
        item[self.label_col] = torch.tensor(label, dtype=torch.long)
        return item

In [15]:
def prepare_dataset(df, tokenizer, text_column="text", label_column="label", max_length=512):
    
    dataset = Dataset.from_pandas(df[[text_column, label_column]])
    
    def tokenize_function(example):
        return tokenizer(
            example[text_column], 
            padding="max_length", 
            truncation=True, 
            max_length=max_length
        )
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset = tokenized_dataset.remove_columns([text_column])
    tokenized_dataset.set_format("torch")

    return tokenized_dataset

In [16]:
def generate_classification_report(model, tokenizer, df, text_column="text", label_column="label", max_length=512, batch_size=32):
    predictions = []
    labels = df[label_column].tolist()
    
    for i in tqdm(range(0, len(df), batch_size)):
        batch_texts = df[text_column].iloc[i:i + batch_size].tolist()
        
        tokens = tokenizer(
            batch_texts,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        
        tokens = {k: v.to(model.device) for k, v in tokens.items()}

        with torch.no_grad():
            outputs = model(**tokens)
            batch_predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
            predictions.extend(batch_predictions)
        del tokens
        torch.cuda.empty_cache()
    
    report = classification_report(labels, predictions, target_names=["Class 0", "Class 1"])
    
    return report

===============================================================================================

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average="weighted")
    # accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")
    
    return {"f1": f1,
            "precision": precision,
            "recall": recall,
           }

In [4]:
device_cpu = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [5]:
# Предобученная модель
model_name = "cointegrated/LaBSE-en-ru"
# model_name = "distilbert/distilbert-base-multilingual-cased"  # https://huggingface.co/distilbert/distilbert-base-multilingual-cased
# model_name = "cointegrated/rubert-tiny"  # https://huggingface.co/cointegrated/rubert-tiny

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/LaBSE-en-ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# print(generate_classification_report(model, tokenizer, test_df, text_column="text", label_column="label"))

In [10]:
train_dataset = TextDataset(train_df, tokenizer, max_length=512)
test_dataset = TextDataset(test_df, tokenizer, max_length=512)
# train_dataset = prepare_dataset(train_df, tokenizer, text_column="text", label_column="label", max_length=512)
# test_dataset = prepare_dataset(test_df, tokenizer, text_column="text", label_column="label", max_length=512)

In [23]:
training_args = TrainingArguments(
    output_dir='./results',                      # Директория для сохранения результатов
    eval_strategy="epoch",                       # Оценка модели на каждой эпохе
    learning_rate=2e-5,                          # Скорость обучения
    per_device_train_batch_size=32,              # Размер батча для обучения
    per_device_eval_batch_size=32,               # Размер батча для валидации
    num_train_epochs=10,                         # Количество эпох
    weight_decay=0.00001,                        # Коэффициент регуляризации
    logging_dir='./logs',                        # Директория для логов (TensorBoard)
    logging_steps=50,                            # Логирование каждые 500 шагов
    load_best_model_at_end=True,                 # Загрузка лучшей модели по окончанию обучения
    save_total_limit=2,                          # Сохраняем только 2 лучшие модели
    metric_for_best_model="f1",                  # Ключевая метрика для выбора лучшей модели
    greater_is_better=True,                      # Лучшая модель — та, где метрика больше
    save_strategy="epoch",                       # Сохраняем модель на каждой эпохе
    report_to="tensorboard",                     # Используем TensorBoard для логирования
    optim="adamw_torch",                         # Явно указываем AdamW как оптимизатор
    warmup_steps=100,
)

In [24]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,                                   # Тренировочный датасет
#     eval_dataset=test_dataset,                                     # Валидационный датасет
#     tokenizer=tokenizer,                                           # Токенизатор
#     compute_metrics=compute_metrics,                               # Функция вычисления метрик
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Ранняя остановка (2 эпохи)
#     # data_collator=data_collator
# )

# # Запуск тренировки
# trainer.train()

In [8]:
model_path = "../model/bert_sentiment_model"

# Сохранение модели
# trainer.save_model(model_path)
# # Сохранение токенизатора
# tokenizer.save_pretrained(model_path)

print(f"Модель сохранена в: {model_path}")

Модель сохранена в: ../model/bert_sentiment_model


### Экспортируем модель в ONNX формат

In [9]:
# Загружаем обученную модель
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()
model = model.to(device_cpu)

In [27]:
print(type(model))
print(type(tokenizer))

<class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [28]:
onnx_path = "../model/onnx/model.onnx"

dummy_input = (
    torch.randint(0, 100, (1, 512)),
    torch.ones((1, 512), dtype=torch.int64)               
)
torch.onnx.export(
    model, dummy_input, onnx_path, 
    input_names=["input_ids", "attention_mask"], output_names=["output"],
    dynamic_axes={
        "input_ids": {0: "batch_size"},
        "attention_mask": {0: "batch_size"},
        "output": {0: "batch_size"}
    },
    opset_version=14
)

print(f"Модель сохранена в {onnx_path}")

Модель сохранена в ../model/onnx/model.onnx


### Экспотируем в OpenVINO и выполняем квантизацию

In [10]:
import openvino as ov
from openvino import convert_model
import nncf
import onnx

In [ ]:
quantized_model_path = "../model/model_openvino/model_openvino.xml"  # Файл для сохранения квантизированной модели
onnx_path = "../model/onnx/model.onnx"
openvino_model_path = "../model/model_openvino/model_openvino.xml"

In [ ]:
onnx_model = onnx.load(onnx_path)

In [ ]:
# Проверка модели и вывод структуры
onnx.checker.check_model(onnx_path)
print(onnx.helper.printable_graph(onnx_path.graph))

In [34]:
model_ir = convert_model(onnx_path)
ov.serialize(model_ir, openvino_model_path)
print(f"Модель сохранена в {openvino_model_path}")

Модель сохранена в ../model/model_openvino/model_openvino.xml


In [35]:
# Загружаем модель OpenVINO
core = ov.runtime.Core()
model_ov = core.read_model(openvino_model_path)
print("Модель загружена!")

Модель загружена!


In [36]:
# import openvino.runtime as ov

# core = ov.Core()
model_ov = core.read_model(openvino_model_path)

for input_node in model_ov.inputs:
    print(f"Input name: {input_node.get_any_name()}, shape: {input_node.get_partial_shape()}")


Input name: input_ids, shape: [?,512]
Input name: attention_mask, shape: [?,512]


In [37]:
# Используем уже загруженный tokenizer
calibration_size = 500
calibration_df = train_df.sample(calibration_size, random_state=42).reset_index(drop=True)

In [38]:
# Создаем DataLoader
calibration_loader = torch.utils.data.DataLoader(
    TextDataset(calibration_df, tokenizer, max_length=512), 
    batch_size=32, 
    shuffle=False
)

In [39]:
# Функция трансформации для калибровки
def transform_fn(data_item):
    input_ids = data_item["input_ids"].numpy()
    attention_mask = data_item["attention_mask"].numpy()
    return {"input_ids": input_ids, "attention_mask": attention_mask}

In [40]:
# Создание калибровочного датасета для NNCF
calibration_dataset = nncf.Dataset(calibration_loader, transform_fn)

In [41]:
sample = next(iter(calibration_dataset.get_inference_data()))
print(sample.keys())  # Должен содержать 'input'
print(sample['input_ids'].shape)  # Должно быть (batch_size, 128)

dict_keys(['input_ids', 'attention_mask'])
(32, 512)


In [ ]:
quantized_model = nncf.quantize(
    model_ov, 
    calibration_dataset, 
    model_type=nncf.ModelType.TRANSFORMER, 
    target_device=nncf.TargetDevice.CPU,
    fast_bias_correction=True,
    preset=nncf.quantization.QuantizationPreset.PERFORMANCE,
    advanced_parameters=nncf.quantization.advanced_parameters.AdvancedQuantizationParameters(
        batchwise_statistics=False
    )    
)

In [ ]:
ov.serialize(quantized_model, 
             "../model/quantized_model/quantized_model.xml", 
             "../model/quantized_model/quantized_model.bin")
print("Квантизация завершена.")

In [1]:
from openvino.runtime import Core

core = Core()
model = core.read_model("../model/bert_quantized_model/quantized_model.xml")

# Проверка, какие типы операций использует модель
op_types = set(op.get_type_name() for op in model.get_ops())
print("Типы операций в модели:", op_types)

# Проверка, есть ли INT8-операции
if any("FakeQuantize" in op or "Quantize" in op or "Dequantize" in op or "Convolution" in op for op in op_types):
    print("Похоже, модель содержит INT8-операции.")
else:
    print("В модели не обнаружено признаков INT8-квантизации.")

Типы операций в модели: {'Divide', 'Reshape', 'FakeQuantize', 'Gather', 'Tanh', 'ShapeOf', 'Unsqueeze', 'Gelu', 'Select', 'Result', 'Add', 'Transpose', 'Multiply', 'Concat', 'Slice', 'MVN', 'Parameter', 'Convert', 'Constant', 'MatMul', 'Subtract', 'Broadcast', 'Sqrt', 'Equal', 'Softmax'}
Похоже, модель содержит INT8-операции.


/usr/lib/python3.10/importlib/util.py:247: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  self.__spec__.loader.exec_module(self)


In [14]:
from openvino.runtime import Core

# Путь к квантизованной модели
model_path = "../model/quantized_model/quantized_model.xml"

# Загружаем модель
core = Core()
model = core.read_model(model_path)

# Получаем все операции модели
ops = model.get_ops()

# Список операций, связанных с квантизацией
int8_related_ops = ["FakeQuantize", "Quantize", "Dequantize", "Convert", "Convolution", "MatMul"]

print("\nINT8-связанные операции в модели:\n")

found = False
for op in ops:
    op_type = op.get_type_name()
    if op_type in int8_related_ops:
        found = True
        print(f"{op_type:15} | Выходной тип: {[out.get_element_type() for out in op.outputs()]}")

if not found:
    print("Не найдено INT8-операций. Возможно, квантизация не была применена.")
else:
    print("\nКвантизация, похоже, успешно применена.")



INT8-связанные операции в модели:

MatMul          | Выходной тип: [<Type: 'float32'>]
Convert         | Выходной тип: [<Type: 'float32'>]
FakeQuantize    | Выходной тип: [<Type: 'float32'>]
MatMul          | Выходной тип: [<Type: 'float32'>]
Convert         | Выходной тип: [<Type: 'float32'>]
FakeQuantize    | Выходной тип: [<Type: 'float32'>]
Convert         | Выходной тип: [<Type: 'float32'>]
Convert         | Выходной тип: [<Type: 'float32'>]
Convert         | Выходной тип: [<Type: 'float32'>]
MatMul          | Выходной тип: [<Type: 'float32'>]
Convert         | Выходной тип: [<Type: 'float32'>]
FakeQuantize    | Выходной тип: [<Type: 'float32'>]
MatMul          | Выходной тип: [<Type: 'float32'>]
MatMul          | Выходной тип: [<Type: 'float32'>]
Convert         | Выходной тип: [<Type: 'float32'>]
FakeQuantize    | Выходной тип: [<Type: 'float32'>]
Convert         | Выходной тип: [<Type: 'float32'>]
Convert         | Выходной тип: [<Type: 'char'>]
MatMul          | Выходной тип:

/usr/lib/python3.10/importlib/util.py:247: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  self.__spec__.loader.exec_module(self)


In [ ]:
import time
import numpy as np
from openvino.runtime import Core

# Пути к моделям
fp32_path = "../model/model_openvino/model_openvino.xml"
int8_path = "../model/bert_quantized_model/quantized_model.xml"

# Создаём runtime
core = Core()
fp32_model = core.compile_model(core.read_model(fp32_path), "CPU")
int8_model = core.compile_model(core.read_model(int8_path), "CPU")

# Имя входного тензора
input_key = next(iter(fp32_model.inputs))

# Получаем нужную форму входа из модели
# input_shape = fp32_model.input(0).shape
# seq_len = input_shape[1]  # обычно 512
seq_len = 512

# Пример входных данных (имитируем батч из 1 примера нужной длины)
dummy_input = np.random.randint(0, 30522, size=(1, seq_len)).astype("int32")

print(f"Generated dummy input shape: {dummy_input.shape}")

# Пример входных данных (имитируем 1 батч токенов, длиной 128)
# dummy_input = np.random.randint(0, 30522, size=(1, 128)).astype("int32")

# Прогрев
_ = fp32_model({input_key: dummy_input})
_ = int8_model({input_key: dummy_input})

# Функция замера
def measure_latency(model, name, runs=100):
    start = time.time()
    for _ in range(runs):
        model({input_key: dummy_input})
    end = time.time()
    avg_latency = (end - start) / runs * 1000  # в мс
    print(f"{name} latency: {avg_latency:.2f} ms")

# Сравнение
measure_latency(fp32_model, "FP32")
measure_latency(int8_model, "INT8")

# Предсказания (1 пример)
fp32_result = fp32_model({input_key: dummy_input})
int8_result = int8_model({input_key: dummy_input})

# Сравнение выходов
fp32_output = list(fp32_result.values())[0]
int8_output = list(int8_result.values())[0]
diff = np.mean(np.abs(fp32_output - int8_output))

print(f"\nСреднее абсолютное отличие между FP32 и INT8: {diff:.6f}")

In [ ]:
from openvino.runtime import Core
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm

# === Инициализация ===
core = Core()
fp32_model = core.compile_model("../model/model_openvino/model_openvino.xml", "CPU")
int8_model = core.compile_model("../model/quantized_model/quantized_model.xml", "CPU")

# Названия входов и выходов
input_keys = [inp.get_any_name() for inp in fp32_model.inputs]
output_key = fp32_model.outputs[0].get_any_name()

# === Функция для предсказания на OpenVINO модели ===
def predict_ov(model, tokenizer, df, max_length=512, batch_size=32):
    predictions = []
    for i in tqdm(range(0, len(df), batch_size)):
        texts = df["text"].iloc[i:i+batch_size].tolist()
        tokens = tokenizer(texts, padding="max_length", truncation=True,
                           max_length=max_length, return_tensors="np")

        # Предсказания
        inputs = {k: tokens[k] for k in input_keys if k in tokens}
        logits = model(inputs)[output_key]
        preds = np.argmax(logits, axis=-1)
        predictions.extend(preds)
    return predictions

# === Получение предсказаний ===
true_labels = test_df["label"].tolist()
preds_fp32 = predict_ov(fp32_model, tokenizer, test_df)
preds_int8 = predict_ov(int8_model, tokenizer, test_df)

# === Сравнение метрик ===
print("FP32:")
print(classification_report(true_labels, preds_fp32, target_names=["Class 0", "Class 1"]))
print("INT8:")
print(classification_report(true_labels, preds_int8, target_names=["Class 0", "Class 1"]))


In [19]:
import psutil
import time
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from openvino.runtime import Core

# === Настройки ===
model_path_fp32 = "../model/model_openvino/model_openvino.xml"
model_path_int8 = "../model/quantized_model/quantized_model.xml"
model_name = "cointegrated/LaBSE-en-ru"
csv_file = "resource_usage_real_input.csv"
batch_size = 16
max_length = 512
n_examples = 32

# === Загрузка моделей ===
core = Core()
fp32_model = core.compile_model(model_path_fp32, "CPU")
int8_model = core.compile_model(model_path_int8, "CPU")
input_keys = [inp.get_any_name() for inp in fp32_model.inputs]
output_key = fp32_model.outputs[0].get_any_name()

# === Загрузка токенизатора и данных ===
tokenizer = AutoTokenizer.from_pretrained(model_name)
from datasets import load_dataset
test_df = pd.DataFrame(load_dataset("imdb")["test"]).sample(n_examples, random_state=42)

# === Токенизация всего батча ===
tokens = tokenizer(
    test_df["text"].tolist(),
    padding="max_length",
    truncation=True,
    max_length=max_length,
    return_tensors="np"
)

# === Разбивка на батчи ===
def get_batches(tokens, batch_size):
    total = len(tokens["input_ids"])
    for i in range(0, total, batch_size):
        yield {k: v[i:i+batch_size] for k, v in tokens.items()}

# === CSV заголовки ===
# with open(csv_file, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["Model", "BatchSize", "AvgLatency_ms", "AvgRAM_MB", "AvgCPU_Percent"])

# === Измерение ===
def measure_with_real_data(model, name):
    all_latencies = []
    all_cpu = []
    all_mem = []
    psutil.cpu_percent(interval=None, percpu=True)

    for batch in tqdm(list(get_batches(tokens, batch_size)), desc=f"{name}"):
        filtered_batch = {k: v for k, v in batch.items() if k in input_keys}
        # Прогрев
        _ = model(filtered_batch)

        start_time = time.time()
        _ = model(filtered_batch)
        end_time = time.time()

        latency = (end_time - start_time) * 1000  # ms
        cpu = psutil.cpu_percent(interval=None, percpu=True)
        mem = psutil.Process().memory_info().rss / (1024 ** 2)

        all_latencies.append(latency)
        all_cpu.append(cpu)
        all_mem.append(mem)

    avg_latency = np.mean(all_latencies)
    avg_ram = np.mean(all_mem)
    # avg_cpu = np.mean(all_cpu)
    num_cpu = psutil.cpu_count()
    len_lst_all_cpu = len(all_cpu)
    avg_cpu = np.zeros(num_cpu)
    for i in range(len_lst_all_cpu):
        for j in range(num_cpu):
            avg_cpu[j] += all_cpu[i][j]
    for i in range(num_cpu):
        avg_cpu[i] = avg_cpu[i] / len_lst_all_cpu

    print(f"\n{name} (реальные данные):")
    # print(all_latencies)
    # for i in all_latencies:
    #     print(f"Время батча {i + 1}: {all_latencies[i]} мс")
    print(f"Среднее время: {avg_latency:.2f} мс")
    # print(all_mem)
    # for i in all_mem:
    #     print(f"Использование RAM для батча {i + 1}: {all_mem[i]} мс")
    print(f"Среднее использование RAM: {avg_ram:.2f} MB")
    for i in avg_cpu:
        print(f"Загрузска CPU: {i:.2f}%")
    # print(avg_cpu)
    # for i in all_cpu:
    #     print(f"Загрузска CPU для батча {i + 1}: {all_cpu[i]} мс")
    # print(f"Средняя загрузка CPU: {avg_cpu:.2f}%")


    # with open(csv_file, mode='a', newline='') as file:
    #     writer = csv.writer(file)
    #     writer.writerow([name, batch_size, avg_latency, avg_ram, avg_cpu])

# === Сравнение ===
measure_with_real_data(fp32_model, "FP32")
measure_with_real_data(int8_model, "INT8")
print(f"\nРезультаты сохранены в {csv_file}")


FP32: 100%|██████████| 2/2 [00:29<00:00, 14.57s/it]



FP32 (реальные данные):
Среднее время: 7273.31 мс
Среднее использование RAM: 3328.04 MB
Загрузска CPU: 19.30%
Загрузска CPU: 21.60%
Загрузска CPU: 25.10%
Загрузска CPU: 26.40%
Загрузска CPU: 98.55%
Загрузска CPU: 97.80%
Загрузска CPU: 99.10%
Загрузска CPU: 98.90%


INT8: 100%|██████████| 2/2 [00:21<00:00, 10.77s/it]


INT8 (реальные данные):
Среднее время: 5252.95 мс
Среднее использование RAM: 5115.29 MB
Загрузска CPU: 22.75%
Загрузска CPU: 25.70%
Загрузска CPU: 29.10%
Загрузска CPU: 30.70%
Загрузска CPU: 98.10%
Загрузска CPU: 98.30%
Загрузска CPU: 98.45%
Загрузска CPU: 97.95%

Результаты сохранены в resource_usage_real_input.csv


In [83]:
print(psutil.cpu_times())
print(psutil.cpu_percent(interval=0.5, percpu=True))
print(psutil.cpu_times_percent())
print(psutil.cpu_count())
print(psutil.cpu_stats())
print(psutil.cpu_freq())
print(psutil.getloadavg())
print(psutil.boot_time())

scputimes(user=11245.06, nice=19.63, system=1437.43, idle=202469.65, iowait=219.24, irq=0.0, softirq=34.18, steal=0.0, guest=0.0, guest_nice=0.0)
[10.2, 10.2, 32.0, 10.0, 13.7, 8.2, 7.8, 7.8]
scputimes(user=7.9, nice=0.0, system=1.6, idle=90.2, iowait=0.2, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0)
8
scpustats(ctx_switches=62233430, interrupts=64095585, soft_interrupts=25131366, syscalls=0)
scpufreq(current=1286.3487499999999, min=800.0, max=4100.0)
(1.14111328125, 1.9072265625, 2.30126953125)
1744785058.0


[4. 5.]
